### Previous code

In [1]:
# Creating dataframe

from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tb = soup.find('table', class_='wikitable')

headings = []
headers = tb.find_all('th')
for i in range(len(headers)):
    headings.append(headers[i].get_text().replace('\n',''))
    
main = []
sub = []
values = tb.find_all('td')

for i in range(len(values)):
    val = values[i].get_text().replace('\n','')
    if i == 0:
        sub.append(val)
    if i%3 != 0:
        sub.append(val)
    if i%3 == 0 and i != 0:
        main.append(sub)
        sub = []
        sub.append(val)
    if i == range(len(values))[-1]:
        main.append(sub)

import pandas as pd
df = pd.DataFrame(columns=headings, data=main)
df_clean = df[df['Borough'] != 'Not assigned']

In [2]:
# Question 2 Answer

coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.columns = ['Postcode', 'Latitude','Longitude']
df_q2 = pd.merge(df_clean, coord, on='Postcode')

### Question 3: EDA

_Explore and cluster the neighborhoods in Toronto. _

In [3]:
df_q2['Borough'].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

Toronoto is split into four areas: Downtown Toronto, Central Toronto, West Toronto and East Toronto

In [4]:
toronto = df_q2[df_q2['Borough'].isin(['Downtown Toronto', 'Central Toronto', 'West Toronto' and 'East Toronto'])]
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [5]:
print('Toronto has', len(toronto['Postcode'].unique()), 'different postcodes')

Toronto has 32 different postcodes


In [6]:
neighbourhoods = list(toronto['Neighbourhood'])
print('Toronto has', len(set(neighbourhoods)), 'different neighbourhoods')

Toronto has 60 different neighbourhoods


In [7]:
toronto[0:5]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


Let's consider the venues in each neighbourhood in Toronto

In [14]:
CLIENT_ID = '5SSF2MVPD3Z31VDMYJBREHIWWHRL4JU2RFY50Z2FDMS41SUN' # your Foursquare ID
CLIENT_SECRET = 'BCG0REZ0XOBSTMQS5CPENWDCKOWIKKLV3UM5L0TFMZ2PENVG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5SSF2MVPD3Z31VDMYJBREHIWWHRL4JU2RFY50Z2FDMS41SUN
CLIENT_SECRET:BCG0REZ0XOBSTMQS5CPENWDCKOWIKKLV3UM5L0TFMZ2PENVG


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
LIMIT = 100

toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
North Toronto West
The Annex
North Midtown
Yorkville
Davisville
Harbord
University of Toronto
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley
Business Reply Mail Processing Centre 969 Eastern


Let's look at the nearby venues in Toronto:

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2933, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [30]:
print('There are {} uniques categories of venue locations.'.format(len(toronto_venues['Venue Category'].unique())))

There are 223 uniques categories of venue locations.


Let's consider how many of each type of venue category is in each neighbourhood on average

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.04,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.0
1,Bathurst Quay,0.0,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
2,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,CN Tower,0.0,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0


We consider the top 5 most frequent types of venue locations in each neighbourhood

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Bathurst Quay----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3     Boat or Ferry  0.07
4  Sculpture Garden  0.07


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2      Steakhouse  0.03
3      Restaurant  0.03
4  Farmers Market  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1    Recording Studio  0.06
2              Garden  0.06
3       Garden Center  0.06
4          Smoke Shop  0.06


----CN Tower----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3     Boat or Ferry  0.07
4  Sculpture Garden  0.07


----Cabbagetown----
                venue  freq
0         

                 venue  freq
0                  Pub  0.14
1          Coffee Shop  0.14
2    Convenience Store  0.07
3  American Restaurant  0.07
4     Sushi Restaurant  0.07


----The Annex----
            venue  freq
0            Café  0.12
1  Sandwich Place  0.12
2     Coffee Shop  0.12
3     Pizza Place  0.08
4  History Museum  0.04


----The Beaches----
               venue  freq
0       Neighborhood   0.2
1  Health Food Store   0.2
2        Coffee Shop   0.2
3               Park   0.2
4                Pub   0.2


----The Beaches West----
               venue  freq
0               Park  0.14
1     Sandwich Place  0.09
2        Pizza Place  0.05
3            Brewery  0.05
4  Fish & Chips Shop  0.05


----The Danforth West----
                    venue  freq
0        Greek Restaurant  0.18
1             Coffee Shop  0.09
2          Ice Cream Shop  0.07
3  Furniture / Home Store  0.05
4      Italian Restaurant  0.05


----Toronto Dominion Centre----
                 venue  freq
0     

We now organise these in order

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

We now add the venue information to the original toronto dataset with clusters to see if they have any effect as features

In [38]:
from sklearn.cluster import KMeans
clustering = toronto.drop(['Postcode', 'Borough', 'Neighbourhood'], 1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(clustering)
labels = kmeans.predict(clustering)

In [39]:
toronto = toronto.assign(Cluster = labels) 
centroids = kmeans.cluster_centers_

Now we consider whether there is any relationship between the types of venues and the assigned clusters

In [47]:
toronto_df = pd.merge(toronto, neighbourhoods_venues_sorted, on='Neighbourhood')

In [46]:
import geopy
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [51]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood'], 
                                  toronto_df['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The clusters seem to be divided by each borough

In [52]:
toronto_df.loc[toronto_df['Cluster'] == 0].bor

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
26,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Garden,Ice Cream Shop,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
27,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Park,Breakfast Spot,Food & Drink Shop,Hotel,Sandwich Place,Burger Joint,Dim Sum Restaurant,Electronics Store,Dumpling Restaurant
28,M5P,Central Toronto,Forest Hill North,43.696948,-79.411307,0,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
29,M5P,Central Toronto,Forest Hill West,43.696948,-79.411307,0,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
30,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Clothing Store,Dessert Shop,Diner,Fast Food Restaurant,Gift Shop,Mexican Restaurant
34,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Coffee Shop,Café,Sushi Restaurant,Restaurant,Pharmacy,Indian Restaurant
37,M4T,Central Toronto,Moore Park,43.689574,-79.383160,0,Gym,Playground,Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
38,M4T,Central Toronto,Summerhill East,43.689574,-79.383160,0,Gym,Playground,Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
42,M4V,Central Toronto,Deer Park,43.686412,-79.400049,0,Coffee Shop,Pub,Light Rail Station,Convenience Store,Sushi Restaurant,Bagel Shop,Sports Bar,Fried Chicken Joint,American Restaurant,Supermarket
